# Análisis de Datos con hanna_ml y hanna_ml.dataframe

Este notebook demuestra el uso de las librerías `hanna_ml` y `hanna_ml.dataframe` para cargar, procesar y analizar datos.

## 1. Importar Librerías Necesarias

Vamos a importar las librerías necesarias para nuestro análisis.

In [ ]:
# Importar hanna_ml y módulos relacionados
import hanna_ml as hml
from hanna_ml.dataframe import DataFrame

# Importar otras librerías útiles para análisis de datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración para visualizaciones
%matplotlib inline
plt.style.use('ggplot')
sns.set(style='whitegrid')

## 2. Cargar Datos con hanna_ml.dataframe

Utilizaremos la clase `DataFrame` de la librería `hanna_ml.dataframe` para cargar y explorar nuestros datos.

In [ ]:
# Cargar datos usando hanna_ml.dataframe
# Podemos crear un DataFrame desde diferentes fuentes

# Ejemplo 1: Crear un DataFrame desde un diccionario
data_dict = {
    'producto_id': [1, 2, 3, 4, 5],
    'nombre': ['Producto A', 'Producto B', 'Producto C', 'Producto D', 'Producto E'],
    'precio': [10.5, 15.75, 8.25, 20.0, 12.5],
    'cantidad': [100, 75, 150, 50, 125]
}

# Crear un DataFrame de hanna_ml
hdf = DataFrame(data_dict)

# Mostrar las primeras filas del DataFrame
print("Primeras filas del DataFrame:")
print(hdf.head())

# Mostrar información del DataFrame
print("\nInformación del DataFrame:")
print(hdf.info())

# Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
print(hdf.describe())

In [ ]:
# Ejemplo 2: Cargar datos desde un archivo CSV (comentado ya que no tenemos un archivo real)

# Supongamos que tenemos un archivo de inventario:
'''
hdf_from_file = DataFrame.from_csv('ruta/a/datos_inventario.csv')
print("\nDatos cargados desde archivo CSV:")
print(hdf_from_file.head())
'''

## 3. Procesar Datos con Funciones de hanna_ml

Ahora vamos a aplicar algunas funciones de procesamiento de datos disponibles en `hanna_ml`.

In [ ]:
# Aplicar transformaciones a los datos usando hanna_ml

# Aplicar una transformación para calcular el valor total (precio * cantidad)
hdf['valor_total'] = hdf['precio'] * hdf['cantidad']

# Mostrar el DataFrame con la nueva columna
print("DataFrame con valor total calculado:")
print(hdf.head())

# Aplicar filtros a los datos
print("\nProductos con precio mayor a 12:")
productos_caros = hdf.filter(lambda row: row['precio'] > 12)
print(productos_caros)

# Ordenar los datos por valor total
print("\nProductos ordenados por valor total (descendente):")
productos_ordenados = hdf.sort('valor_total', ascending=False)
print(productos_ordenados)

# Aplicar una transformación con map
print("\nAplicar descuento del 10% a los precios:")
hdf['precio_con_descuento'] = hdf['precio'].map(lambda x: x * 0.9)
print(hdf.head())

In [ ]:
# Realizar análisis estadístico con hanna_ml

# Calcular correlaciones entre variables numéricas
print("\nMatriz de correlación:")
correlacion = hml.correlation(hdf, ['precio', 'cantidad', 'valor_total'])
print(correlacion)

# Calcular estadísticas agrupadas
print("\nEstadísticas por rango de precio:")

# Crear una función para categorizar los productos por precio
def categorizar_precio(precio):
    if precio < 10:
        return 'económico'
    elif precio < 15:
        return 'medio'
    else:
        return 'premium'

# Agregar columna de categoría
hdf['categoria'] = hdf['precio'].map(categorizar_precio)

# Agrupar por categoría y calcular estadísticas
estadisticas_por_categoria = hml.group_by(hdf, 'categoria', 
                                        aggregations=['mean', 'sum'],
                                        columns=['precio', 'cantidad', 'valor_total'])
print(estadisticas_por_categoria)

## 4. Visualizar Resultados

Vamos a crear algunas visualizaciones para entender mejor nuestros datos.

In [ ]:
# Convertir el DataFrame de hanna_ml a pandas para visualización
pandas_df = hdf.to_pandas()

# Gráfico de barras para mostrar el valor total por producto
plt.figure(figsize=(10, 6))
sns.barplot(x='nombre', y='valor_total', data=pandas_df)
plt.title('Valor Total por Producto')
plt.xlabel('Producto')
plt.ylabel('Valor Total ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Gráfico de dispersión para mostrar la relación entre precio y cantidad
plt.figure(figsize=(8, 6))
sns.scatterplot(x='precio', y='cantidad', size='valor_total', hue='categoria', data=pandas_df)
plt.title('Relación entre Precio y Cantidad')
plt.xlabel('Precio ($)')
plt.ylabel('Cantidad en Inventario')
plt.tight_layout()
plt.show()

# Gráfico de pastel para mostrar la distribución de productos por categoría
plt.figure(figsize=(8, 8))
categoria_counts = pandas_df['categoria'].value_counts()
plt.pie(categoria_counts, labels=categoria_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Distribución de Productos por Categoría de Precio')
plt.axis('equal')
plt.tight_layout()
plt.show()

## 5. Aplicar Modelos de Machine Learning con hanna_ml

Vamos a demostrar cómo utilizar algunos modelos de machine learning proporcionados por `hanna_ml`.

In [ ]:
# Preparar datos para un modelo de regresión
# Predecir el valor total basado en precio y cantidad

# Crear más datos sintéticos para el ejemplo
import random
random.seed(42)

# Crear datos adicionales
data_extended = {
    'producto_id': list(range(1, 101)),
    'precio': [random.uniform(5, 25) for _ in range(100)],
    'cantidad': [random.randint(10, 200) for _ in range(100)]
}

# Crear DataFrame extendido
hdf_extended = DataFrame(data_extended)

# Calcular valor_total (variable objetivo)
hdf_extended['valor_total'] = hdf_extended['precio'] * hdf_extended['cantidad']

# Dividir en conjuntos de entrenamiento y prueba
from hanna_ml.model_selection import train_test_split

X = hdf_extended[['precio', 'cantidad']]
y = hdf_extended['valor_total']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar un modelo de regresión
from hanna_ml.regression import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)

# Calcular métricas de evaluación
from hanna_ml.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Error cuadrático medio: {mse:.2f}")
print(f"Coeficiente de determinación (R²): {r2:.4f}")

# Visualizar predicciones vs valores reales
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
plt.xlabel('Valor Total Real')
plt.ylabel('Valor Total Predicho')
plt.title('Predicciones vs Valores Reales')
plt.tight_layout()
plt.show()

# Mostrar los coeficientes del modelo
print(f"\nCoeficientes del modelo:")
print(f"Intercepto: {model.intercept_:.4f}")
print(f"Coeficiente para precio: {model.coef_[0]:.4f}")
print(f"Coeficiente para cantidad: {model.coef_[1]:.4f}")

## Conclusión

En este notebook hemos visto cómo utilizar las librerías `hanna_ml` y `hanna_ml.dataframe` para:

1. Cargar y explorar datos
2. Realizar transformaciones y manipulaciones de datos
3. Aplicar análisis estadístico
4. Visualizar resultados 
5. Entrenar y evaluar modelos de machine learning

Estas librerías proporcionan una interfaz intuitiva y potente para realizar análisis de datos y aplicar técnicas de machine learning a tus conjuntos de datos.